Setup

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import torch
import CNN
import numpy as np

# from config_plots import global_settings
# global_settings()
import warnings
warnings.filterwarnings("ignore")

Hyper parameters

In [2]:
# defining global variables
experimentName = "test_22987_suffix50_11_dataLoader3_pretrained_depth_resnet50_augmentation"
useRandomSearch = False
numOfExperiments=50

from configParser import ConfigParser, getModelName
hyperpSearchObject = ConfigParser(experimentName).getHyperpSearchObject()

import os
experimentName = os.path.join(experimentName,"hyperp-search")
import TrialStatistics
ts = TrialStatistics.TrialStatistics(experimentName)

import dataLoader3 as dataLoader

Cuda support

In [3]:
# CUDA support 
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

We are using cuda


Define objective function

In [4]:
import os
from torch import nn
from sklearn.metrics import f1_score

all_experiment_params = []
datasetManager = dataLoader.datasetManager(experimentName)
def objective(experiment_params):
    experiment_params["numOfTrials"] = experiment_params["numOfTrials"] or 1
    
    print("experiment: ", experiment_params)
    all_experiment_params.append(experiment_params)
    
    # load images
    datasetManager.updateParams(experiment_params)
    dataset = datasetManager.getDataset()
    numberOfSpecies = len(dataset.getSpeciesList())
    numberOfGenus = len(dataset.getGenusList())

    for i in range(experiment_params["numOfTrials"]):
        trialName = os.path.join(experimentName, getModelName(experiment_params, i))

        # Train/Load model
        architecture = {
            "species": numberOfSpecies,
            "genus" : numberOfGenus
        }
        model = CNN.create_model(architecture, experiment_params)
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        if os.path.exists(CNN.getModelFile(trialName)):
            CNN.loadModel(model, trialName)
            print("Model {0} loaded!".format(trialName))
        else:
            loss_list, accuracy_list, epochs, time_elapsed = CNN.trainModel(train_loader, 
                                                                            validation_loader, 
                                                                            experiment_params, 
                                                                            model, trialName)
        
        # Update trial outcomes for statistics
        predlist, lbllist = CNN.getLoaderPredictions(test_loader, model)
        ts.addTrialPredictions(experiment_params, predlist, lbllist, numberOfSpecies)
        macro_f1 = f1_score(lbllist.cpu(), predlist.cpu(), average='macro')

        ts.addTrial(experiment_params,
                    {'loss': CNN.getCrossEntropyFromLoader(test_loader, model),
                     'accuracy': CNN.getAccuracyFromLoader(test_loader, model),
                     'macro f1': macro_f1,
                     'time': time_elapsed,
                     'epochs': epochs
                    }, i)
                
    answer ={
        'loss': ts.getStatistic(experiment_params, 'loss', 'mean'),
        'loss-std': ts.getStatistic(experiment_params, 'loss', 'std'),
        'time': ts.getStatistic(experiment_params, 'time', 'mean'),
        'time-std': ts.getStatistic(experiment_params, 'time', 'std'),
        'epochs': ts.getStatistic(experiment_params, 'epochs', 'mean'),
        'epochs-std': ts.getStatistic(experiment_params, 'epochs', 'std'),
        'accuracy': ts.getStatistic(experiment_params, 'accuracy', 'mean'),
        'accuracy-std': ts.getStatistic(experiment_params, 'accuracy', 'std'),
        'macro-f1': ts.getStatistic(experiment_params, 'macro f1', 'mean'),
        'macro-f1-std': ts.getStatistic(experiment_params, 'macro f1', 'std'),
        'status': STATUS_OK,}
    
    return {**experiment_params, **answer}

Iterate

In [5]:
from hyperopt import fmin, hp, STATUS_OK, Trials, space_eval, plotting, rand, tpe
import pickle

trials = Trials()
bestLoss = fmin(objective, 
                        space=hyperpSearchObject, 
                        algo=rand.suggest if useRandomSearch == False else tpe.suggest, 
                        trials=trials,
                        max_evals=numOfExperiments)

experiment:                                           
{'batchSize': 450, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'kernelSize': 5, 'kernels': (16, 32), 'learning_rate': 0.00033880729163409263, 'n_channels': 3, 'n_epochs': 1000, 'numOfTrials': 2, 'patience': 250, 'suffix': '50_11', 'training_count': 0.64, 'useNormalization': False, 'usePretrained': True, 'useZCAWhitening': False, 'validation_count': 0.16}
Loading dataset...                                    
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

  0% (3 of 550) |                        | Elapsed Time: 0:00:00 ETA:   0:00:18

Going through image files


100% (550 of 550) |######################| Elapsed Time: 0:00:39 Time:  0:00:39


dataset has a mean: tensor([0.4503, 0.4249, 0.3802]) and std: tensor([0.3063, 0.3081, 0.3048])
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_dataLoader3_pretrained_depth_resnet50_augmentation/hyperp-search/tc0.640000_vc0.160000_d224_c3_zcaFalse_nFalse_ptTrue/dataset.pkl written
using a pretrained resnet50 model...
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_dataLoader3_pretrained_depth_resnet50_augmentation/hyperp-search/trainingIndex.pkl written
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_dataLoader3_pretrained_depth_resnet50_augmentation/hyperp-search/valIndex.pkl written
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_dataLoader3_pretrained_depth_resnet50_augmentation/hyperp-search/testIndex.pkl written
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_dataLoader3_pretrained_depth_resnet50_augmentation/hyperp-search/tc0.640000_vc0.160000_d224_c3_zcaFalse_nFalse_ptTrue/trainingLoader.pkl written
file /

...

N/A% (0 of 1000) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training started...


 95% (953 of 1000) |################### | Elapsed Time: 11:11:31 ETA:   0:32:51


  2%|▏         | 1/50 [35:14:25<1726:47:07, 126865.88s/trial, best loss: 0.6013216972351074]


KeyboardInterrupt: 

analyze

In [ ]:
ts.showStatistics()

In [ ]:
ts.showStatistics(False)

In [ ]:


import pandas as pd
from IPython.display import display, HTML

# bestParams = space_eval(hyperpSearchObject, bestLoss)
# print("best params = ", pd.DataFrame(bestParams, index=[0]))
best_trial = sorted(trials.results, key=lambda x: x['loss'], reverse=False)[0]
print("Best trial")
display(HTML(pd.DataFrame(best_trial, index=[0]).to_html()))

# save trials
pickle.dump(trials, open(os.path.join(experimentName,"trials.p"), "wb"))
ts.saveStatistics()
ts.saveStatistics(False)

Generate confusion matrices

In [ ]:
for experiment_params in all_experiment_params:    
    print("experiment: ", experiment_params)
    
    datasetManager.updateParams(experiment_params)
    dataset = datasetManager.getDataset()
    speciesList = dataset.getSpeciesList()
    ts.printTrialConfusionMatrix(experiment_params, speciesList, True)
    ts.printF1table(experiment_params, dataset)